<a href="https://colab.research.google.com/github/sayaleepande/GenAI/blob/main/M5_Lab1_LangChain_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="background: linear-gradient(135deg, #001a70 0%, #0055d4 100%); color: white; padding: 25px; text-align: center; border-radius: 10px; margin-bottom: 20px;">
  <h1 style="font-size: 32px; margin-bottom: 10px;">🧠 LangChain Lab 1: Introduction</h1>
  <p style="margin: 0; font-size: 16px;">Welcome to the first hands-on lab for LangChain! In this lab, we will:</p>
  <p style="margin-top: 10px; font-size: 18px; font-weight: bold;">Instructor: Dr. Dehghani</p>
  <p style="margin-top: 5px; font-size: 14px;">
    Learn more at <a href="https://langchain.com" target="_blank" style="color: #ffdd57; text-decoration: underline;">LangChain Website</a>
  </p>
</div>

<div style="background: #f0f5ff; border-radius: 12px; padding: 20px; border: 1px solid #0055d4;">
  <p style="line-height: 1.6; font-size: 16px; margin-bottom: 15px;">
    LangChain is a framework for building applications powered by large language models (LLMs).  
    It provides modular components—such as prompt templates, memory, and chains—that make it easy to develop, test, and deploy LLM-based solutions.
  </p>
  <h2 style="color: #0055d4; margin-top: 0; font-size: 24px; padding-bottom: 10px; border-bottom: 2px solid #0055d4;">Lab Objectives</h2>
  <ul style="line-height: 1.8; font-size: 16px; margin: 0; padding-left: 20px;">
    <li>Set up LangChain in Google Colab</li>
    <li>Interact with OpenAI models</li>
    <li>Work with open-source models like Falcon</li>
  </ul>
</div>


In [1]:
# ⚙️ Install only the essential packages for LangChain with OpenAI & Gemini support

!pip install -q --upgrade langchain                # Core LangChain framework for building LLM workflows
!pip install -q --upgrade langchain-community      # Community integrations (still useful for many non-OpenAI/Gemini models)
!pip install -q --upgrade langchain-openai         # ✅ NEW: Dedicated package for OpenAI integrations
!pip install -q --upgrade langchain-google-genai   # Integration for Google's Gemini models
!pip install -q --upgrade openai                   # OpenAI SDK for native API calls (not strictly needed for LangChain, but often useful)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.4/438.4 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 30.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.4/720.4 kB 19.3 MB/s eta 0:00:00


## 🔑 Step 2: Set Up OpenAI API Key
If you want to use OpenAI models like GPT-4, you need an API key. Run the code below and enter your key when prompted.

In [2]:
# ⚙️ Load API Keys from Colab Secrets
# ==================================

import os                                  # Used to set environment variables for API keys
from google.colab import userdata          # To securely access stored secrets in Colab

# Retrieve your stored secrets (API keys)
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')   # OpenAI API key for GPT models
GEMINI_API_KEY = userdata.get('Gemini_api_key')   # Google Gemini API key for Gemini models

# Set environment variables for the APIs and confirm success
if OPENAI_API_KEY:
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY   # Set OpenAI key as environment variable
    print("✅ OpenAI API key loaded successfully!")
else:
    print("❌ OpenAI API key not found. Please set 'OPENAI_API_KEY' in Colab secrets.")

if GEMINI_API_KEY:
    os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY   # Set Gemini key as environment variable
    print("✅ Google Gemini API key loaded successfully!")
else:
    print("❌ Google Gemini API key not found. Please set 'GEMINI_API_KEY' in Colab secrets.")


✅ OpenAI API key loaded successfully!
✅ Google Gemini API key loaded successfully!


<div style="background: linear-gradient(135deg, #001a70 0%, #0055d4 100%); color: white; padding: 25px; text-align: center; border-radius: 10px; margin-bottom: 20px;">
  <h1 style="font-size: 32px; margin-bottom: 10px;">🤖 LLM Connection Check (via LangChain)</h1>
  <p style="margin: 0; font-size: 16px;">
    This step tests if <strong>LangChain</strong> can communicate with your selected language model—such as OpenAI, Gemini, or others.<br>
    All messaging with the LLM is handled by LangChain, giving you a flexible and unified workflow.
  </p>
</div>

<div style="background: #f0f5ff; border-radius: 12px; padding: 18px; margin-bottom: 22px; border: 1px solid #0055d4;">
  <ol style="line-height: 1.7; font-size: 16px; margin: 0; padding-left: 20px;">
    <li><strong>We call the language model through LangChain</strong> — not directly. LangChain manages the connection and delivers the response. 🌐</li>
    <li>If you get a valid reply, your setup is good to go for any LLM provider! 🎯</li>
  </ol>
</div>

<div style="background: #ffffff; border-radius: 12px; padding: 18px; border: 1px solid #0055d4; margin-bottom: 22px;">
  <h2 style="color: #0055d4; font-size: 22px; margin-top: 0; margin-bottom: 10px;">🔹 How <code>.invoke()</code> Works</h2>
  <p style="font-size: 16px; margin-bottom: 12px;">
    The <code>.invoke()</code> method is the main way to send prompts (messages) to a language model in LangChain.<br>
    You wrap your input in a <code>HumanMessage</code> (or <code>SystemMessage</code>, etc.), then pass it as a list.<br>
    LangChain sends the message to the LLM and returns a response object.
  </p>
  <pre style="background: #f7faff; border-radius: 8px; padding: 12px; font-size: 15px; border: 1px solid #cce0ff;">
response = llm.invoke([HumanMessage(content="Summarize LangChain in one sentence.")])
print(response.content)  # ⬅️ This gives you the model's reply
  </pre>
  <p style="font-size: 15px; margin: 0;">
    <strong>Tip:</strong> This workflow is the same for all LLM providers supported by LangChain!
  </p>
</div>


In [3]:
# =========================================================
# 🌟 LangChain Connection Test: OpenAI (Generalized)
# =========================================================

# 🚩 Check available OpenAI models at: https://platform.openai.com/docs/models/
#   (Recommended: 'gpt-3.5-turbo', 'gpt-4o', 'gpt-4-turbo', etc.)

# 1️⃣ Import the correct ChatOpenAI class (from the updated package!)
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

# 2️⃣ Set your desired model (update to any available OpenAI chat model as needed)
OPENAI_MODEL = "gpt-3.5-turbo"    # <-- Change this string to any OpenAI chat model you have access to

# 3️⃣ Initialize the LLM connection
llm = ChatOpenAI(
    model=OPENAI_MODEL,
    api_key=OPENAI_API_KEY
)

# 4️⃣ Define your prompt
prompt = "What is LangChain in one short sentence?"

# 5️⃣ Invoke the model (send the prompt as a HumanMessage)
response = llm.invoke([HumanMessage(content=prompt)])

# 6️⃣ Display the response
print("🔹 OpenAI Response:", response.content)


🔹 OpenAI Response: LangChain is a decentralized platform for language learning using blockchain technology.


In [4]:
# =======================================
# 🧐 Inspecting the LLM Response Object
# =======================================

# 🏷️ Print the type of the response object
print("🔖 Type of response:", type(response))

# 🧩 Print the raw response object (see all attributes/methods)
print("\n🔎 Raw response object:\n")
response


🔖 Type of response: <class 'langchain_core.messages.ai.AIMessage'>

🔎 Raw response object:



AIMessage(content='LangChain is a decentralized platform for language learning using blockchain technology.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 16, 'total_tokens': 29, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BbvMaXhFliUqMcBWBPnqoyXJjSGrf', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--1721cd85-4f15-4c18-9cc9-bbfb88e88fbe-0', usage_metadata={'input_tokens': 16, 'output_tokens': 13, 'total_tokens': 29, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [9]:
# ✋ **Hands-On: Reading the LLM's Response**
# ============================================

# Replace '-----' in the placeholders with the correct method or key to retrieve the requested information.

# Task 1: Get the main content from the LLM response
response_text = response.content  # Extract the main response text (e.g., "content" for ChatOpenAI)
print("Main response text:", response_text)

# Task 2: Get the number of prompt tokens used
prompt_tokens = response.usage_metadata['input_tokens']  # Extract the number of tokens used in the input prompt
print("Number of prompt tokens used:",prompt_tokens )

# Task 3: Get the number of response tokens generated
response_tokens = response.usage_metadata['output_tokens']  # Extract the number of tokens used in the generated response
print("Number of response tokens generated:", response_tokens)


Main response text: LangChain is a decentralized platform for language learning using blockchain technology.
Number of prompt tokens used: 16
Number of response tokens generated: 13


<div style="background: linear-gradient(135deg, #001a70 0%, #0055d4 100%); color: white; padding: 22px 28px 16px 28px; border-radius: 12px; margin-bottom: 18px; text-align: center;">
  <h2 style="font-size: 26px; margin-bottom: 8px;">💬 Multi-Turn Conversation in LangChain</h2>
  <p style="font-size: 16px; margin-bottom: 0;">
    A <strong>multi-turn conversation</strong> allows an AI to retain context across multiple exchanges, making interactions more natural and intelligent.<br>
    Instead of treating each query independently, the AI builds on previous inputs, improving coherence and accuracy. <span style="font-size: 22px;">🤝</span>
  </p>
</div>

<div style="background: #f0f5ff; border-radius: 12px; padding: 18px; margin-bottom: 22px; border: 1px solid #0055d4;">
  <h3 style="color: #0055d4; font-size: 20px; margin-top: 0;">✨ Why Use Multi-Turn Conversations?</h3>
  <ul style="font-size: 16px; margin: 0; padding-left: 20px; line-height: 1.7;">
    <li><strong>Context Retention</strong> – AI remembers past interactions, leading to more relevant responses.</li>
    <li><strong>Realistic Dialogue</strong> – Mimics human conversations, making chatbots more engaging.</li>
    <li><strong>Improved Accuracy</strong> – Responses are refined based on earlier exchanges.</li>
    <li><strong>Scalable Design</strong> – Supports long-form discussions without losing context.</li>
  </ul>
  <p style="font-size: 15px; margin-top: 12px;">
    <em>This approach is ideal for applications like <b>financial advisors, chatbots, research assistants</b>, and other AI-driven services that require ongoing, dynamic conversations.</em>
  </p>
</div>


In [10]:
# 🌟 Multi-Turn Conversation Example: Coffee History (Concise Responses)
# =======================================================================

# Import message classes: SystemMessage (AI behavior), HumanMessage (user input), AIMessage (AI responses)
from langchain.schema import SystemMessage, HumanMessage, AIMessage

# Instruct the AI to act as a coffee historian and reply in one short sentence
messages = [
   SystemMessage(content="You are a coffee historian. Provide concise, one-sentence answers."),
   HumanMessage(content="What is the origin of coffee?"),
   AIMessage(content="Coffee was first discovered in the Kaffa region of Ethiopia in the 9th century."),
   HumanMessage(content="How did coffee spread beyond Ethiopia?"),
   AIMessage(content="It traveled via trade routes into Yemen and then throughout the Ottoman Empire."),
   HumanMessage(content="When did coffee reach Europe?")
]

# Send the structured messages to your initialized LangChain model
response = llm.invoke(messages)
print(response.content)  # Expect a one-sentence response on coffee reaching Europe

Coffee reached Europe in the 17th century, with the first coffeehouse opening in Venice in 1645.


In [12]:
# ✋ **Hands-On: Completing a Multi-Turn Conversation (Travel Assistant)**
#==========================================================================

# 📌 Task Instructions:
# - Below is a conversation with a **travel assistant** AI.
# - Fill in the last `HumanMessage` with a relevant travel-related question.
# - Complete the placeholder `response = ----(----)` to correctly call the LLM.

# Define a conversation with missing parts
messages = [
    SystemMessage(content="You are a helpful travel assistant, providing recommendations for destinations and travel tips."),
    HumanMessage(content="What are some must-visit places in Japan?"),
    AIMessage(content="Some must-visit places in Japan include Tokyo, Kyoto, and Osaka. Each city offers unique cultural and historical experiences."),
    HumanMessage(content="Suggest a city to travel during Spring season which has history and culture")  #  Task: Fill in a relevant follow-up question
]

# 🔧 Task: Complete the function to generate a response from the LLM
response = llm.invoke(messages)  # Call the LLM correctly using the messages

print(response.content)  # Display the AI's response


Kyoto is a perfect choice for a springtime visit, as it is known for its beautiful cherry blossoms and traditional Japanese culture. With its historic temples, serene gardens, and picturesque streets, Kyoto offers a rich cultural experience amidst the vibrant colors of spring.


Observation: The AI response seems correct. It was great to see it mention the city would be good during the springtime too (the context to which was included in the HumanMessage)..

<div style="background: linear-gradient(135deg, #001a70 0%, #0055d4 100%); color: white; padding: 20px; border-radius: 10px; margin-bottom: 18px;"> <h2 style="margin: 0 0 10px 0;">🔹 What is a Multi-LLM Model?</h2> <p style="margin: 0;"> A <strong>Multi-LLM model</strong> is a setup where you can send the same question or conversation to multiple large language models (LLMs)—for example, OpenAI's GPT-4 and Google's Gemini—at the same time. This allows you to compare their answers, pick the best response, or even blend their strengths for more reliable results. </p> </div> <div style="background: #f0f5ff; border-radius: 12px; padding: 16px; border: 1px solid #0055d4;"> <h3 style="color: #0055d4; margin: 0 0 8px 0;">Why Does LangChain Make This Easy?</h3> <ul style="padding-left: 20px; margin: 0; font-size: 16px;"> <li><b>Unified Interface:</b> LangChain lets you connect to many LLM providers (OpenAI, Google, Hugging Face, and more) using the same simple code.</li> <li><b>Modular Chaining:</b> You can send the same messages or workflows to any model without rewriting your logic for each one.</li> <li><b>Rapid Experimentation:</b> Instantly compare outputs, performance, or reliability from different models—helping you choose the right LLM for your app or research.</li> </ul> </div> <div style="background: #fff3f3; border-radius: 12px; padding: 12px; border: 1px solid #ff6b6b; color: #d32f2f;"> <b>In short:</b> <i>LangChain empowers you to use, compare, and combine multiple AI models in one unified workflow—making your LLM projects more flexible and future-proof.</i> </div>

In [15]:
# ✨ Multi-LLM Comparison: Gemini vs. ChatGPT (Coffee in 2050)
# ==========================================================================

# 📝 Imports explained:
from langchain.schema import SystemMessage, HumanMessage, AIMessage   # For chat message formatting (system/user/AI)
from langchain_google_genai import ChatGoogleGenerativeAI             # Google's Gemini LLM via LangChain
from langchain.chat_models import ChatOpenAI                          # OpenAI models (ChatGPT, GPT-4) via LangChain
import pandas as pd                                                   # For tabular storage and display

# 🟦 System and User Setup: Future Coffee Culture
messages = [
    SystemMessage(content="You are a fun futurist AI barista. Predict and explain what coffee culture will look like in the year 2050."),
    HumanMessage(content="Describe the biggest change in how people enjoy coffee in 2050 in few sentnene."),
]

# ☕ Initialize LLMs (update model names as needed)
gemini_llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", temperature=0.7)
chatgpt_llm = ChatOpenAI(model_name="gpt-4", temperature=0.7)

# 🤖 Generate responses from both models
gemini_response = gemini_llm.invoke(messages)
chatgpt_response = chatgpt_llm.invoke(messages)

# 📊 Store responses in a DataFrame for easy comparison and future use
df = pd.DataFrame({
    "Model": ["Gemini", "ChatGPT"],
    "Response": [gemini_response.content, chatgpt_response.content]
})

# 🖨️ Print the DataFrame as a table (plain text)
pd.set_option('display.max_colwidth', None)
display(df)


,Model,Response
0,Gemini,"Forget your morning latte run! In 2050, personalized, on-demand coffee is the norm. Hyper-personalized brews, tailored to your biometrics and mood via smart-home systems and delivered instantly via a sleek, countertop coffee-synthesizer, will replace the traditional cafe experience for many. Sustainability will be paramount, with hyperlocal, lab-grown beans and zero-waste brewing processes."
1,ChatGPT,"In 2050, coffee culture has been revolutionized with the advent of the ""Personalized Coffee Experience."" This innovation uses AI and advanced biotechnology to tailor each coffee blend to an individual's taste preferences, health requirements, and even mood states. More than just a beverage, coffee has become an immersive, multi-sensory experience, integrating elements of augmented reality to add visual appeal, personalized narratives, and even virtual coffee tasting tours."


In [ ]:
# =============================================================
# 📊 LLM Math Reasoning Challenge: Gemini vs. ChatGPT (Lab Style)
# =============================================================

import time
import matplotlib.pyplot as plt   # For plotting response times
import pandas as pd               # For tabular storage and display

from langchain.schema import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chat_models import ChatOpenAI

# -------------------------------------------------------------
# 📝 Math Problem Prompt: Can LLMs Reason Like Mathematicians?
# -------------------------------------------------------------
prompt = (
    "Solve for x in the equation: 3x^3 + 5x^2 - 12x + 7 = 21. "
    "Highlight answers by ---[ x1=  ] ---"
)

# -------------------------------------------------------------
# 🤖 Initialize LLMs: Gemini & ChatGPT (Low Temp for Accuracy)
# -------------------------------------------------------------
gemini_llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.3)
chatgpt_llm = ChatOpenAI(model_name="gpt-4", temperature=0.3)

# -------------------------------------------------------------
# ⏱️ Get LLM Responses & Measure Response Time
# -------------------------------------------------------------
start_gemini = time.time()
gemini_response = gemini_llm.invoke([HumanMessage(content=prompt)])
end_gemini = time.time()
gemini_time = end_gemini - start_gemini

start_chatgpt = time.time()
chatgpt_response = chatgpt_llm.invoke([HumanMessage(content=prompt)])
end_chatgpt = time.time()
chatgpt_time = end_chatgpt - start_chatgpt

# -------------------------------------------------------------
# 📋 Store Model Outputs & Timing in a Table
# -------------------------------------------------------------
responses = pd.DataFrame({
    "Model": ["Gemini", "ChatGPT"],
    "Response": [gemini_response.content, chatgpt_response.content],
    "Response Time (s)": [gemini_time, chatgpt_time]
})

# 🖥️ Display table for easy side-by-side review
display(responses)

# -------------------------------------------------------------
# 📈 Visualize: Response Time Comparison (Brand Colors)
# -------------------------------------------------------------
plt.figure(figsize=(6,4))
plt.bar(
    responses["Model"],
    responses["Response Time (s)"],
    color=["#00E3FF", "#10A37F"]  # Gemini blue, OpenAI green
)
plt.ylabel("Response Time (seconds)", fontsize=12)
plt.title("LLM Math Problem Response Time", fontsize=14, color="#0055d4")
plt.grid(axis='y', linestyle='--', alpha=0.3)
plt.show()

# -------------------------------------------------------------
# 🧐 Notes & Analysis:
#   - The equation 3x^3 + 5x^2 - 12x + 7 = 21 has three real solutions:
#         x = -1,   x ≈ 1.852,   x ≈ -2.519
#   - Review the 'Response' column above: Does the LLM show clear reasoning?
#   - Did the model provide a step-by-step solution, not just the final answer?
# -------------------------------------------------------------


In [ ]:
# ============================================================
# 🧑‍💻 Hands-On: Exploring LLM Prompt Engineering & Model Choice
# ============================================================

"""
Task:
1. Choose a different math equation (e.g., a different cubic, a system of equations, or even a word problem).
2. Experiment with your prompt style! For example:
    - Ask the LLM to "show every step"
    - Use ReAct: "First, reflect on what is needed, then solve, then verify your solution"
    - Ask for self-evaluation: "Explain why your answer makes sense"
3. Try at least TWO different LLMs (e.g., Gemini, ChatGPT, or another you have access to).
4. Record response content and timing in a DataFrame.
5. At the end, **write down your observations**:
    - Did the prompt style or LLM change the accuracy or reasoning?
    - Which model did better? Was ReAct or self-checking effective?
"""


In [ ]:
# ============================================================
# 🔎 Gemini Model Explorer: List Top 10 Newest Models
# ============================================================

# Import the Gemini model listing function
from google.generativeai import list_models
import pandas as pd

# ------------------------------------------------------------
# 🚀 Fetch & Display: Newest Gemini Models (Top 10)
# ------------------------------------------------------------
gemini_data = []
try:
    model_list = list(list_models())
    # 🗂️ Sort by model name in reverse (often, higher = newer)
    model_list_sorted = sorted(model_list, key=lambda m: m.name, reverse=True)
    for m in model_list_sorted[:10]:  # Show only the top 10
        gemini_data.append({
            "Model Name": m.name,
            "Supported Methods": ", ".join(m.supported_generation_methods)
        })
    df_gemini = pd.DataFrame(gemini_data)

    print("🔵 === Top 10 Newest Gemini Models ===")
    display(df_gemini)

    if not gemini_data:
        print("⚠️ No Gemini models found. Your API key may not have access, or Gemini is not available in your region/account.")

except Exception as e:
    print("❌ Error:", e)


In [ ]:
# ============================================================
# 🔵 OpenAI Model Explorer: List Newest Models & Metadata
# ============================================================

import os
import pandas as pd
from openai import OpenAI

# ------------------------------------------------------------
# 🗝️ Load OpenAI API Key from Environment
# ------------------------------------------------------------
openai_key = os.environ.get("OPENAI_API_KEY", None)
model_data = []

if openai_key:
    try:
        # 🤖 Connect to OpenAI and retrieve models
        client = OpenAI(api_key=openai_key)
        # 📋 Sort models by name/id in reverse (newest first)
        models = sorted(client.models.list().data, key=lambda m: m.id, reverse=True)
        for m in models[:15]:  # Show up to 15 models
            model_data.append({
                "Model Name": m.id,
                "Owned By": getattr(m, "owned_by", ""),
                "Created": getattr(m, "created", ""),
                "Object": getattr(m, "object", "")
                # Note: Purpose/Description not present in API!
            })
        df = pd.DataFrame(model_data)
        print("🔵 === Newest OpenAI Models: Metadata ===")
        display(df)
        if not model_data:
            print("⚠️ No OpenAI models found. Your API key may not have access, or your account is limited.")
    except Exception as e:
        print("❌ Error checking OpenAI API key:", str(e))
else:
    print("⚠️ No OpenAI API key found in environment.")


<div style="background: linear-gradient(135deg, #001a70 0%, #0055d4 100%); color: white; padding: 24px; border-radius: 12px; margin-bottom: 20px;">
  <h2 style="margin-top:0; font-size: 26px;">🤗 Introduction: Hugging Face & Classic LLMs</h2>
  <p style="font-size: 16px;">
    <strong>Hugging Face</strong> is an open-source platform that hosts thousands of ready-to-use machine learning models, making it easy to experiment with and deploy powerful language models locally or in the cloud.<br>
    In this section, we'll compare two popular Hugging Face models:
  </p>
  <ul style="font-size: 16px; margin: 18px 0 8px 18px;">
    <li><span style="font-size: 20px;">🤖</span> <b>GPT-2</b> – One of the first large language models to generate human-like text. It set the stage for the modern LLM boom, and remains a classic benchmark.</li>
    <li><span style="font-size: 20px;">⚡</span> <b>DistilGPT2</b> – A smaller, faster, and more efficient version of GPT-2. It offers nearly the same capabilities but with lighter resource requirements, making it ideal for quick prototyping.</li>
  </ul>
  <p style="font-size: 15px;">
    We'll see how each model responds to creative prompts and compare their outputs side by side!
  </p>
</div>


In [ ]:
# ============================================================
# 🛠️ Setup: Install & Import Required Packages for HF Pipelines
# ============================================================

!pip install -q --upgrade langchain langchain-huggingface transformers

from langchain_huggingface import HuggingFacePipeline   # Wraps Hugging Face models for LangChain
from transformers import pipeline                       # Builds local text-generation pipelines
from langchain.schema import HumanMessage               # Formats chat/user messages for LLMs


In [ ]:
# =============================================================
# 🤗☕ Compare Local Hugging Face LLMs: GPT-2 vs. DistilGPT2
# =============================================================

import pandas as pd  # For tabular output

model_ids = [
    ("🤖 GPT-2", "gpt2"),
    ("⚡ DistilGPT2", "distilgpt2"),
]

prompt = "What will be the future of AI in 2050?!"

responses = []

for model_name, model_id in model_ids:
    # 'pipeline' builds a ready-to-use text-generation model from Hugging Face with a single command
    hf_pipe = pipeline("text-generation", model=model_id, max_new_tokens=250)
    llm = HuggingFacePipeline(pipeline=hf_pipe)
    response = llm.invoke([HumanMessage(content=prompt)])
    responses.append({"Model": model_name, "Response": response.strip()})

# Save and display the DataFrame of responses
df_response = pd.DataFrame(responses)
print("🔵 Model responses to the prompt:")
display(df_response)


In [ ]:
# =============================================================
# 🧪 Hands-On: Getting Better Answers from DistilGPT2
# =============================================================

# Background:
# DistilGPT2 is a smaller, faster version of GPT-2.
# While it's efficient, it often just repeats the prompt or gives very generic answers.
# This happens because it isn't specifically trained to follow instructions or answer questions directly.

# Your Task:
# 1. Try using DistilGPT2 for an open-ended question (e.g., "What will be the future of AI in 2050?").
# 2. Observe if it repeats the prompt or gives an unhelpful answer.

# To improve its output, experiment with these techniques:
#   - Make your prompt longer or more explicit (e.g., "Q: What is the future of AI in 2050?\nA:").
#   - Add clear instructions or a separator (e.g., "Answer in 2 sentences: ...").
#   - Enable sampling and set temperature higher (e.g., do_sample=True, temperature=0.8).
#   - Try using the full GPT-2 model for comparison.

# For each approach:
# - Record what happens. Does the model's output improve? When does it still repeat or ignore the prompt?
# - Compare your best DistilGPT2 output to GPT-2’s response.
# - Summarize your observations and submit them as your exercise report.


<div style="font-family: Arial, sans-serif; max-width: 600px; margin: 0 auto; line-height: 1.6; color: #333;">
  <h2 style="color: #0055d4; border-bottom: 2px solid #0055d4; padding-bottom: 8px;">
    The Leap from GPT-2 to Today’s Super-Models
  </h2>
  <p>
    Back in 2019, <strong>GPT-2</strong> shook the world with its 1.5 billion parameters (and even a distilled 82 million version) by generating surprisingly coherent paragraphs from web-scraped text. It proved that transformers could model language at scale—but still struggled with long-form reasoning, factual accuracy, and open-ended prompts.
  </p>
  <p>
    Its lighter sibling, <strong>DistilGPT2</strong>, was optimized for speed and efficiency but often echoes the prompt or produces very basic completions, especially for creative or complex questions. This is a common limitation with small, “vanilla” language models that haven’t been tuned for instruction or dialogue.
  </p>
  <p>
    Fast forward to today: modern LLMs boast tens or hundreds of billions of parameters, trained on diverse, multilingual data and fine-tuned with human feedback. They not only write essays and debug code, but also follow instructions, explain reasoning, and adapt to specialized domains. The leap from “sometimes impressive, often generic” to “genuinely useful and insightful” has happened in just a few years—a testament to the rapid evolution of generative AI.
  </p>
</div>


<div style="background: linear-gradient(135deg, #f0f5ff 0%, #e6f0ff 100%); border-radius: 12px; padding: 25px; margin-bottom: 25px; border: 1px solid #0055d4;">
  <h2 style="color: #0055d4; margin-top: 0; font-size: 26px; padding-bottom: 10px; border-bottom: 2px solid #0055d4;">🦅 Falcon: High-Performance Open-Source Large Language Model</h2>
  <p style="font-size: 16px; color: #222; line-height: 1.7; margin-bottom: 18px;">
    <b>Falcon</b> is a state-of-the-art, open-source large language model (LLM) developed by the <a href="https://www.tii.ae/" target="_blank" style="color:#0055d4;text-decoration:underline;"><b>Technology Innovation Institute (TII)</b></a> in Abu Dhabi. Falcon stands out for its high efficiency and performance, and is widely adopted in both research and industry.
  </p>
  <ul style="margin-bottom: 18px; font-size: 16px;">
    <li>🚀 Available in multiple sizes: <b>Falcon-7B</b> and <b>Falcon-40B</b></li>
    <li>⚡ Optimized for low memory usage and fast inference</li>
    <li>🛠️ Supports a broad range of NLP tasks (text generation, summarization, chat, and more)</li>
    <li>🌐 Fully open-source with community support on Hugging Face</li>
  </ul>
  <div style="margin-bottom: 10px; font-size: 16px;">
    <b>Useful Resources:</b>
    <ul>
      <li><a href="https://huggingface.co/tiiuae/falcon-7b" target="_blank" style="color:#0055d4;">Falcon-7B Model Card (Hugging Face)</a></li>
      <li><a href="https://github.com/tiiuae/falcon-llm" target="_blank" style="color:#0055d4;">Falcon GitHub Repository</a></li>
    </ul>
  </div>
  <div style="background: #e6f0ff; border-radius: 8px; padding: 14px 18px; margin-top: 16px; border-left: 5px solid #0055d4;">
    <b>Why Falcon?</b> <br>
    Falcon models are a powerful choice for building efficient, scalable AI applications, and serve as a strong open-source alternative to proprietary LLMs.
  </div>
</div>


In [ ]:
"""
# =============================================================
# 🦾 Falcon-7B: Local Model Loading & Pipeline Initialization
# =============================================================

- Loads the Falcon-7B large language model and tokenizer using Hugging Face `transformers`.
- Sets up a text-generation pipeline that runs efficiently on your Colab or local GPU (using bfloat16 precision and auto device selection).
- This pipeline lets you generate human-like text locally—no internet or external API needed once the model is downloaded.
"""

from transformers import AutoTokenizer, pipeline
import torch

# Specify the model name (Falcon-7B)
model = "tiiuae/falcon-7b"

# Load the tokenizer for the Falcon-7B model
tokenizer = AutoTokenizer.from_pretrained(model)

# Build a text-generation pipeline for efficient local inference
text_pipeline = pipeline(
    "text-generation",           # Task: generate text
    model=model,                 # Model to use
    tokenizer=tokenizer,         # Tokenizer to use
    torch_dtype=torch.bfloat16,  # Use bfloat16 for memory efficiency (if supported)
    device_map="auto",           # Automatically use GPU if available
)


> ### ⚡️ How Does This Cell Work?
>
> - The code imports 🤗 `transformers` and loads the Falcon-7B model directly into your Colab session.
> - Downloading and setting up the model **can take several minutes** (due to the large size and initial processing).
> - **Falcon-7B is an open-source model**: you don't need any API key, and all text generation happens locally on your Colab GPU—no outside cloud service required!


In [ ]:
"""
# =============================================================
# ☕🦾 Falcon-7B CalmMindBot: AI Multi-Turn Wellness Demo
# =============================================================
"""

prompt = (
    "CalmMindBot is a compassionate AI assistant who supports people with stress and emotional wellness."
    "\nUser: Hi CalmMindBot, I’ve been feeling overwhelmed lately. What’s a simple way to relax?"
    "\nCalmMindBot:"
)

sequences = text_pipeline(
    prompt,
    max_new_tokens=600,        # Allow for several conversation turns
    do_sample=True,
    top_k=15,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

for seq in sequences:
    full_text = seq['generated_text']
    # Split at each role's tag (User: or CalmMindBot:)
    import re
    dialogue_lines = re.split(r'(User:|CalmMindBot:)', full_text)
    # The split leaves ['', 'User:', '...', 'CalmMindBot:', '...', ...]
    # So we iterate in steps of 2, skipping any empty initial
    for i in range(1, len(dialogue_lines), 2):
        speaker = dialogue_lines[i].strip()
        content = dialogue_lines[i+1].strip()
        if speaker == "User:":
            print(f"👤 User: {content}\n")
        elif speaker == "CalmMindBot:":
            print(f"🤖 CalmMindBot: {content}\n")


> ### 🤔 How Does a Transformer Model Like Falcon-7B Generate Text?
>
> - The `transformers` library loads the Falcon-7B model’s weights into your computer or Colab GPU.
> - When you give it a prompt, the model **processes each word (token) step by step**, multiplying learned weights (from its neural network) to predict the most likely next word.
> - This process repeats, one token at a time, until the model completes its response—creating answers that seem natural and relevant.
>
> In short: you provide a starting message, and the model uses its “learned math” (weights) to generate a reply, token by token!


In [ ]:
# ✋ **Hands-On: Querying Falcon-7B with LangChain**
# Replace the placeholders to:
# 1️⃣ Initialize the Hugging Face model correctly.
# 2️⃣ Use the correct method to query the model.
# 3️⃣ Store the response in the right variable.

from langchain_huggingface import HuggingFaceEndpoint

# ✅ Step 1: Initialize the Falcon model
llm_falcon = ----- (repo_id="tiiuae/falcon-7b-instruct")  # 🔧 Replace '-----' with the correct class

# ✅ Step 2: Define the question
question = "How will AI impact the job market in the next decade?"

# ✅ Step 3: Query the model
response_falcon = llm_falcon.----- (question)  # 🔧 Replace '-----' with the correct method

# ✅ Display the response
print("🔹 Falcon-7B Response:", response_falcon)


## 🎉 Congratulations!

You have successfully completed the **Introduction to LangChain** lab. 🚀  
We hope you found it insightful and are excited to explore more!  

💡 **Next Steps:**  
- Try using different LLMs in LangChain.  
- Experiment with structured vs. plain text prompts.  
- Explore advanced features like memory and chains.

Happy Coding! 💻✨  
